# Topic Modeling and Gibbs Sampling

Задача: описать текст через распределение весов по некоторому фиксированному набору топиков (тегов). Например, для набора тегов Политика, Военные сражения, Спорт, Интернет, Драма представить роман "Война и мир" как вектор (0.3, 0.2, 0, 0, 0.5), а статью в газете про допинг в велоспорте как вектор (0.1, 0, 0.7, 0, 0.2).

Для чего, например, это нужно: имея векторное представление для текстов, тексты можно сравнивать, рекомендовать похожие.

Условие: даны только набор текстов и количество тем.

## Немного теории

Будем представлять текст как неупорядоченный набор слов (Bag-of-words model). Предположим, что имеется K тегов и для каждого тега выбрано распределение $\phi_k$ над списком всевозможных слов (словарем из N слов). По сути, каждое $\phi_k$ - это вектор длины N из неотрицательных величин, в сумме дающих 1. Вектора $\phi_k$ независимы и моделируются распредеделением Дирихле $Dir(\beta)$. Теперь, чтобы собрать текст d из $n$ слов, будем действовать по следующей схеме:


* выберем распредление для тегов $\theta_d$. Вновь, $\theta_d$ - это вектор длины K из неотрицательных величин, в сумме дающих 1. Поэтому естественно брать $\theta_d \sim Dir(\theta | \alpha)$

* Для i от 1 до n:
  * выберем тег $z_i$ согласно распределению $\theta_d$
  * выберем слово $w_i$ из распределения для данного тега, т.е. $w_i \sim \phi_{z_i}$
  * добавляем слово $w_i$ в текст.

Полученная модель называется моделью LDA (Latent Dirichlet Allocation). Описанная  схема задает совместное распределение скрытых и наблюдаемых параметров по всем текстам корпуса размера M в виде:


$p(\textbf{w}, \textbf{z}, \theta, \phi | \alpha, \beta) = Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)$.

Здесь $\textbf{w}$ и $\textbf{z}$ обозначают вектора слов и тегов по всем текстам, $\theta$ - набор из $\theta_d$ для каждого документа (матрица $M\times K$), $\phi$ - набор из $\phi_k$ для каждого тега (матрица $K\times N$).


![img](https://www.researchgate.net/publication/336065245/figure/fig1/AS:807371718815752@1569503826964/Latent-Dirichlet-allocation-LDA-process-and-its-two-outputs-a-LDA-document.ppm)

Наша задача - восстановить распределение $p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)$.


Немного упростим жизнь, и поставим себе задачей восстановить распределение $p(\textbf{z} | \textbf{w}, \alpha, \beta) = \int\int p(\textbf{z}, \theta, \phi | \textbf{w}, \alpha, \beta)\textrm{d}\theta\textrm{d}\phi$.

В этот момент на помощь приходить алгоритм Gibbs Sampling. Напомним, для оценки набора парамеров $\textbf{z} = (z_1, z_2, ..., z_m)$ используется схема:

$z_i^{(t)} \sim p(z_i^{(t)}\ | \ z_1=z_1^{t}, ..., z_{i-1}=z_{i-1}^{t},
z_{i+1}=z_{i+1}^{t-1}, z_{m}=z_{m}^{t-1})$.

Условные распределения выводятся так. Сначала замечаем, что

$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) = \frac{p(z_i,\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}  = \frac{p(\textbf{z}, \textbf{w}| \alpha, \beta)}{p(\textbf{z}_{\hat{i}}, \textbf{w}| \alpha, \beta)}$.

Здесь $\textbf{z}_{\hat{i}}$ - вектор без $i$-oй копмоненты. 

Далее расписываем:

$p(\textbf{z}, \textbf{w}| \alpha, \beta) = \int\int p(\textbf{z}, \textbf{w}, \theta, \phi| \alpha, \beta)\textrm{d}\theta\textrm{d}\phi = \int\int Dir(\theta | \alpha) Dir(\phi|\beta)Cat(\textbf{z}|\theta)Cat(\textbf{w}|\phi_z)\textrm{d}\theta\textrm{d}\phi = 
\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta \int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi$

и обнаруживаем, что оба интеграла в последнем выражении вычисляются аналитически. Для примера первый:

$\int Dir(\theta | \alpha) Cat(\textbf{z}|\theta)\textrm{d}\theta = \prod\limits_d \int Dir(\theta_d | \alpha) Cat(\textbf{z}_d|\theta_d)\textrm{d}\theta_d = \prod\limits_d \int \frac{1}{B(\alpha)}\prod\limits_k \theta_{d, k}^{\alpha-1}\prod\limits_i \theta_{d, z_i}\textrm{d}\theta_d = \prod\limits_d\frac{1}{B(\alpha)}\int\prod\limits_k \theta_{d, k}^{n_{d, k} + \alpha - 1}\textrm{d}\theta_d = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)}$.

Здесь  $n_{d,k}$ - количество тэгов $k$ в тексте $d$, $n_{d,\cdot}$ - вектор длины $K$ из этих величин.

Аналогично, второй интеграл 
$\int Dir(\phi|\beta)Cat(\textbf{w}|\phi_z)\textrm{d}\phi = \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$,

где $n_{k,\cdot}$ - вектор длины $N$ встречаемости слов внутри тэга $k$.

Получаем: 

$p(\textbf{z}, \textbf{w}| \alpha, \beta) =\prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(\alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(\beta)}$.

Теперь
$p(z_i|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) = \prod\limits_d \frac{B(n_{d,\cdot} + \alpha)}{B(n_{d,\cdot}^{\hat{i}} + \alpha)} \prod\limits_k \frac{B(n_{k,\cdot} + \beta)}{B(n_{k,\cdot}^{\hat{i}} + \beta)}$.

Выражение упрощается дальше, расписывая бета-функцию через гамма-функции. Напомним,
$B(x_1, ..., x_k) = \frac{\Gamma(x_1)\cdot...\cdot\Gamma(x_k)}{\Gamma(x_1 + ... + x_k)}$, а также $\Gamma(n) = (n-1)\Gamma(n-1)$. Получим:

$p(z_i=s|\textbf{z}_{\hat{i}}, \textbf{w}, \alpha, \beta) \propto (n_{d, s} + \alpha_s - 1) \frac{n_{s, w_i} + \beta_s - 1}{\sum\limits_{w}(n_{s, w} + \beta_{w}) - 1}$.

Знак $\propto$ означает пропорциональность с точностью до коэффициентов, независящих от $i$, $s$.


С этого места можно полностью собрать алгоритм моделирования плотности $p(\textbf{z}| \textbf{w}, \alpha, \beta)$. Введем  обозначение $n_k$ - количество слов, отнесенных к тегу $k$, $W$ - общее количество слов в корпусе, $\overline\beta=\sum\limits_w\beta_w$.

Алгоритм:

* заведем счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* случайным образом расставим теги словам, обновим счетчики $n_{k, w}$, $n_{d, k}$, $n_k$
* пока не сойдемся к стационарному режиму:
  * для каждого $i$ от 1 до $W$:
      * для каждого $k$ от 1 до $K$:
        * вычисляем $p(z_i=k|...) = (n_{d, k} + \alpha_k - 1) \frac{n_{k, w_i} + \beta_k - 1}{n_k + \overline\beta - 1}$
        * сэмплим новый $z_i$ из полученного распределения $p(z_i=k|...)$




Восстановив распредление для $\textbf{z}$, можем оценить $\theta$ и $\phi$, о которых мы ненадолго забыли. Оценить можно, например, через матожидание по апостериорным распределениям. Получите формулы самостоятельно!

Литература:

http://u.cs.biu.ac.il/~89-680/darling-lda.pdf

https://www.cs.cmu.edu/~mgormley/courses/10701-f16/slides/lecture20-topic-models.pdf

Перейдем к практике.

## Датасет

Возьмем популярный датасет [20 Newsgroups](http://qwone.com/~jason/20Newsgroups/), встроенный в пакет ```sklearn```. Датасет состоит из ~20К текстов, классифицированных на 20 категорий. Датасет разбит на ```train``` и ```test```. Для загрузки используем  модуль ```fetch_20newsgroups```, в параметрах указать, что мета информацию о тексте загружать не нужно:

In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [2]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

Атрибут ```traget``` хранит номера категорий для текстов из обучающей выборки:

In [3]:
newsgroups_train.target[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

Доступ к самим текстам через атрибут ```data```. Выведем текст и категорию случайного примера из обучающего датасета:

In [4]:
n = 854
print('Topic = {0}\n'.format(newsgroups_train.target_names[newsgroups_train.target[n]]))
print(newsgroups_train.data[n])

Topic = rec.motorcycles

hey... I'm pretty new to the wonderful world of motorcycles... I just
bought
a used 81 Kaw KZ650 CSR from a friend.... I was just wondering what kind of

saddle bags I could get for it (since I know nothing about them)  are there
bags for the gas tank?  how much would some cost, and how much do they
hold?
thanks for your advice!!!  I may be new to riding, but I love it
already!!!!
:)




## Векторное представление текста

Представим текст как вектор индикаторов вхождений слов из некоторого словаря в текст. Это простейшая модель BOF. 

Сформируем словарь на основе нашего набора текстов. Для этого используем модуль ```CountVectorizer```:

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words=frozenset({'a', 'about', 'above', 'across', 'after',
                                      'afterwards', 'again', 'against', 'all',
                                      'almost', 'alone', 'along', 'already',
                                      'also', 'although', 'always', 'am',
                                      'among', 'amongst', 'amoungst', 'amount',
                                      'an', 'and', 'another', 'any', 'anyhow',
                                      'anyone', 'anything', 'anyway',
                                      'anywhere', ...}),
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

Количество проиндексированных слов:

In [6]:
len(vectorizer.vocabulary_)

101322

Проиндексированные слова и их индексы:

In [7]:
vectorizer.vocabulary_

{'wondering': 96879,
 'enlighten': 37256,
 'car': 25717,
 'saw': 80420,
 'day': 31927,
 'door': 34741,
 'sports': 84312,
 'looked': 57247,
 'late': 55606,
 '60s': 9843,
 'early': 35902,
 '70s': 11174,
 'called': 25437,
 'bricklin': 24108,
 'doors': 34742,
 'really': 76269,
 'small': 83208,
 'addition': 16806,
 'bumper': 24583,
 'separate': 81450,
 'rest': 77676,
 'body': 23430,
 'know': 54493,
 'tellme': 87913,
 'model': 62594,
 'engine': 37208,
 'specs': 84050,
 'years': 99608,
 'production': 73174,
 'history': 46690,
 'info': 49800,
 'funky': 41874,
 'looking': 57250,
 'mail': 59071,
 'fair': 39296,
 'number': 66680,
 'brave': 23973,
 'souls': 83779,
 'upgraded': 92389,
 'si': 82337,
 'clock': 27889,
 'oscillator': 68519,
 'shared': 81848,
 'experiences': 38637,
 'poll': 72039,
 'send': 81378,
 'brief': 24125,
 'message': 60923,
 'detailing': 33127,
 'procedure': 73122,
 'speed': 84088,
 'attained': 20236,
 'cpu': 30233,
 'rated': 75904,
 'add': 16791,
 'cards': 25769,
 'adapters': 1

Индекс, например, для слова anyone:

In [8]:
vectorizer.vocabulary_.get('car')

25717

А теперь преобразуем строку в вектор:

In [9]:
text = 'I was wondering if anyone out there could enlighten me on this car I saw'
x = vectorizer.transform([text])

Какой тип имеет объект, на который указывает ```x```?

In [10]:
type(x)

scipy.sparse.csr.csr_matrix

Разреженная матрица!

### Отступление про разреженные матрицы

Список ненулевых элементов матрицы:

In [11]:
x.data

array([1, 1, 1, 1])

Индексы строк и столбцов для ненулевых элементов:

In [12]:
x.nonzero()

(array([0, 0, 0, 0], dtype=int32),
 array([25717, 37256, 80420, 96879], dtype=int32))

Преобразование к объекту ndarray (именно после приведения к такому виду разреженные матрицы можно подставлять в функции, например, библиотеки Numpy):

In [13]:
x.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

Вернемся к словарю. Раскодируем вектор ```x``` в список слов:

In [14]:
vectorizer.inverse_transform(x)

[array(['car', 'enlighten', 'saw', 'wondering'], dtype='<U81')]

Пропало слово ```I```. Но дело в том, что по умолчанию ```CountVectorizer``` отбрасывает последовательности, короче 2 символов. На это указывает параметр ```token_pattern='(?u)\\b\\w\\w+\\b'```.

Переведем весь набор текстов обучающего датасета в набор векторов, получим матрицу ```X_train```:

In [15]:
X_train = vectorizer.fit_transform(newsgroups_train.data)
X_train.shape

(11314, 101322)

О пользе разреженных матриц. Отношение числа ненулевых элементов ко всем элементам матрицы ```X_train```:

In [16]:
X_train.nnz / np.prod(X_train.shape)

0.0006593137467596179

Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы. 

In [83]:
def lda(X, n_topics, alpha, beta, n_iter=10):
    n_kw = np.zeros((n_topics, X.shape[1]))
    n_dk = np.zeros((X.shape[0], n_topics))
    n_k = np.zeros(n_topics)
    
    docs, words = X.nonzero()
    z = np.random.choice(n_topics, len(docs))
    
    for doc, word, cur_z in zip(docs, words, z):
        n_dk[doc, cur_z] += 1
        n_kw[cur_z, word] += 1
        n_k[cur_z] += 1
    
    for cur_iter in tqdm(range(n_iter)):
        for i in range(len(docs)):
            cur_word = words[i]
            cur_doc = docs[i]
            cur_topic = z[i]
            
            n_dk[cur_doc, cur_topic] -= 1
            n_kw[cur_topic, cur_word] -= 1
            n_k[cur_topic] -= 1
            
            p = (n_dk[cur_doc, :] + alpha - 1) * (n_kw[:, cur_word] + beta - 1) / \
                (n_k + beta.sum() - 1)
            z[i] = np.random.choice(np.arange(n_topics), p=p / p.sum())
            
            n_dk[cur_doc, z[i]] += 1
            n_kw[z[i], cur_word] += 1
            n_k[z[i]] += 1
    
    return z, n_kw, n_dk, n_k

In [84]:
n_topics = 20
z, n_kw, n_dk, n_k = lda(X_train, n_topics, 2 * np.ones(n_topics), \
                         2 * np.ones(n_topics), 20)

100%|██████████| 20/20 [13:12<00:00, 39.60s/it]


_Выведем топ 10 слов для кажой из найденных тем_

In [103]:
top_words = np.argsort(n_kw, axis=1)[:, -10:]

for topic in range(20):
    doc = np.zeros((1, X_train.shape[1]))
    for word in top_words[topic]:
        doc[0, word] = 1
    print('Topic {}:\t{}'.format(topic, '\t'.join(vectorizer.inverse_transform(doc)[0])))

Topic 0:	address	edu	fax	help	information	mail	new	phone	research	university
Topic 1:	available	don	key	keys	number	real	systems	time	use	used
Topic 2:	believe	bible	christian	does	don	god	jesus	people	say	true
Topic 3:	air	article	earth	launch	moon	nasa	orbit	sp	space	years
Topic 4:	does	don	just	know	like	need	problem	thanks	use	work
Topic 5:	actually	does	got	just	know	like	people	really	thing	things
Topic 6:	best	email	interested	mail	new	offer	price	sale	thanks	used
Topic 7:	code	file	files	ftp	program	thanks	using	version	window	windows
Topic 8:	based	fact	general	include	information	number	public	related	research	subject
Topic 9:	bike	car	don	good	just	know	like	make	right	time
Topic 10:	14	34	m4	ml	mq	mr	mt	mw	tm	wm
Topic 11:	believe	better	don	fact	good	like	people	say	think	way
Topic 12:	don	game	games	good	play	players	season	team	win	year
Topic 13:	did	don	edu	good	just	know	like	think	time	years
Topic 14:	children	government	history	israel	israeli	jews	killed	people	war	wo

Можем увидеть соответствие некоторых найденных тем темам из датасета
 - Тема 2 - soc.religion.christian
 - Тема 3 - sci.space
 - Тема 7 - comp.windows.x
 - Тема 10 - comp.os.ms-windows.misc
 - Тема 16 - talk.politics.guns
 - Тема 14 - talk.politics.mideast
 
LDA достаточно хорошо справился с поиском тем.